# Sacred Experiment Comparison

The goal of this notebook is to provide a mechanism to compare sacred experiments: the cfg, the metrics, and the outcome.

In [40]:
# project imports
import env
from utils import read_json_data, compare_dicts
from models import MODEL_DIR


# python & package imports
from collections import defaultdict
import pandas as pd
import json
import os


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Model Leaderboard

In [54]:
scores = dict()
model_dir = os.path.join(MODEL_DIR, 'voxel_vaegan1', 'modelnet10')
for m in os.listdir(model_dir):
    name = m
    metrics_json = os.path.join(model_dir, m, 'metrics.json')
    #print(metrics_json)
    #if os.path.exists(metrics_json):
    #    df = pd.read_json(metrics_json, orient='values')
    #    break
    metrics_dict = read_json_data(metrics_json)
    if metrics_dict:
        # assuming only keys are epoch\d+
        keys = [int(key.replace('epoch', '')) for key in metrics_dict.keys()]
        last_epoch = sorted(keys)[-1]
        last_key = 'epoch{}'.format(last_epoch)
        scores[name] = metrics_dict[last_key]
        scores[name]['epoch'] = last_epoch

df = pd.DataFrame.from_dict(scores, orient='index').sort_values('enc_loss')
df

,dis_loss,reconstruction_loss,kl_divergence,epoch,dec_loss,enc_loss,elapsed_time
2019-03-12_09-30-02,-999.000000,8.277367,-0.000000,499,-999.000000,0.080834,3.687410
10k_10epochs1,-999.000000,1243.016235,0.016579,9,-999.000000,0.379853,124.788061
2019-03-12_05-28-52,-999.000000,1253.438232,0.006444,49,-999.000000,0.382728,4.481774
animals_100epochs,968.545166,13196.958984,49.095032,99,754.960266,5.555391,34.820713
2019-03-12_09-44-48,-999.000000,8.127342,0.000017,499,-999.000000,7.936874,3.661042
2019-03-12_09-38-41,-999.000000,8.545296,-0.000000,499,-999.000000,8.345016,3.693110
best,-999.000000,808.499634,488.078369,9,-999.000000,261.985138,43.024081
2019-03-12_10-22-19,-999.000000,30.710892,0.000220,49,-999.000000,299.911255,34.678700


In [39]:
scores = dict()
model_dir = os.path.join(MODEL_DIR, 'voxel_vaegan1')
for m in os.listdir(model_dir):
    name = m
    metrics_json = os.path.join(model_dir, m, 'metrics.json')
    df = pd.read_json(metrics_json)
    break
    metrics_dict = read_json_data(metrics_json)
    if metrics_dict:
        # assuming only keys are epoch\d+
        keys = [int(key.replace('epoch', '')) for key in metrics_dict.keys()]
        last = 'epoch{}'.format(sorted(keys)[-1])
        scores[name] = metrics_dict[last]

        
        
bests = defaultdict(lambda: defaultdict(tuple))
for name, values in scores.items():
    for k, v in values.items():
        curr = ('unknown', 999999) if len(bests[k]) == 0 else bests[k]
        bests[k] = (name, v) if v < curr[1] else curr

for k, v in bests.items():
    print('{}:\t\t{}'.format(k, v))

{'2019-03-10_14-32-41': {'dis_loss': 393.0194396972656, 'reconstruction_loss': 1343.6868896484375, 'kl_divergence': 0.010936439037322998, 'dec_loss': 267.1690673828125, 'enc_loss': 0.4103938043117523, 'elapsed_time': 7.001890126864115}, '2019-03-12_09-44-48': {'dis_loss': -999.0, 'reconstruction_loss': 8.127342224121094, 'kl_divergence': 1.6748905181884766e-05, 'dec_loss': -999.0, 'enc_loss': 7.9368743896484375, 'elapsed_time': 3.661041820049286}, '2019-03-12_05-28-52': {'dis_loss': -999.0, 'reconstruction_loss': 1253.438232421875, 'kl_divergence': 0.006444036960601807, 'dec_loss': -999.0, 'enc_loss': 0.38272830843925476, 'elapsed_time': 4.481773793697357}, 'animals_100epochs': {'dis_loss': 968.545166015625, 'reconstruction_loss': 13196.958984375, 'kl_divergence': 49.09503173828125, 'dec_loss': 754.9602661132812, 'enc_loss': 5.555391311645508, 'elapsed_time': 34.820713206132254}, '2019-03-12_09-38-41': {'dis_loss': -999.0, 'reconstruction_loss': 8.545295715332031, 'kl_divergence': -0.0

## Config Comparison

In [5]:
# provide the full path to root dir of the experiment output (the parent dir of the ckpts)
exp1 = os.path.join(MODEL_DIR, 'voxel_vaegan1/best')
exp2 = os.path.join(MODEL_DIR, '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-14_15-18-38')

In [13]:
cfg1 = read_json_data(os.path.join(exp1, 'cfg.json'), verbose=True)
cfg2 = read_json_data(os.path.join(exp2, 'cfg.json'), verbose=True)
assert isinstance(cfg1, dict)
assert isinstance(cfg2, dict)
compare_dicts(cfg1, cfg2)


/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-14_15-18-38/cfg.json does not exist


AssertionError: 

## Just in case you need to retrieve a ckpt_dir from an old sacred config.json save

In [10]:
root = '/home/jcworkma/jack/3d-form/notebooks/experiments_vaegan'
for exp_dir in os.listdir(root):
    if exp_dir == '_sources':
        continue
    #print('-------')
    #print(exp_dir)
    cfg_path = os.path.join(root, exp_dir + '/config.json')
    with open(cfg_path, 'r') as json_file:
        cfg = json.load(json_file)
    ckpt_dir = cfg.get('cfg', 'banana').get('model', 'apple').get('ckpt_dir', 'orange')
    #print(ckpt_dir)
    if '2019-03-12_16-36-25' in ckpt_dir:
        print('found it!')
        print(cfg_path)
        print(ckpt_dir)
        break

found it!
/home/jcworkma/jack/3d-form/notebooks/experiments_vaegan/99/config.json
voxel_vaegan1/2019-03-12_16-36-25
